In [15]:
data = {
    "datetime": [
        "2024-07-09 12:44:04",
        "2024-07-09 12:44:04",
        "2024-07-09 13:17:26",
        "2024-07-09 13:17:26",
        "2024-07-11 11:05:32",
        "2024-07-11 14:09:06",
        "2024-07-11 14:09:06",
    ],
    "side": ["BUY", "BUY", "SELL", "SELL", "SELL", "BUY", "BUY"],
    "quantity": [25, 125, 50, 100, 25, 10, 15],
    "buy_price": [89.0, 89.0, 89.0, 89.0, 0.0, 2.5, 2.5],
    "buy_amount": [2225.0, 11125.0, 4450.0, 8900.0, 0.0, 25.0, 37.5],
    "sell_price": [0.0, 0.0, 117.1, 117.1, 4.3, 0.0, 0.0],
    "sell_amount": [0.0, 0.0, 5855.0, 11710.0, 107.5, 0.0, 0.0],
    "holding_price": [89.0, 89.0, 89.0, 0.0, 4.3, 5.5, 0.0],
    "holding_quantity": [25, 150, 100, 0, -25, -15, 0],
    "holding_amount": [2225.0, 13350.0, 8900.0, 0.0, -107.5, -82.5, 0.0],
    "pnl_amount": [0, 0, 1405, 2810, 0, 0, -45],
}
df = pd.DataFrame(data)

In [16]:
import pandas as pd
from common_utilities import Portfolio, global_path


def main():
    # read the csv file
    df_TradeHistory = pd.read_csv(global_path.tradehistory_silver_file_path)
    # Convert 'datetime' to datetime type
    df_TradeHistory["datetime"] = pd.to_datetime(df_TradeHistory["datetime"])
    # sort the dataframe by date
    df_TradeHistory = df_TradeHistory.sort_values(by="datetime")
    # replace scrip code to compnay name
    df_Symbol = pd.read_csv(global_path.symbol_silver_file_path)
    df_Symbol["scrip_code"] = df_Symbol["scrip_code"].astype(str)
    # Merge df_TradeHistory with df_Symbol on the matching columns
    df_TradeHistory = df_TradeHistory.merge(
        df_Symbol[["scrip_code", "symbol"]],
        left_on="scrip_code",
        right_on="scrip_code",
        how="left",
    )
    # Assign the new column 'stock_name' in df_TradeHistory to the values from 'symbol'
    df_TradeHistory["stock_name"] = df_TradeHistory["symbol"].combine_first(df_TradeHistory["stock_name"])
    df_TradeHistory = df_TradeHistory[df_TradeHistory["stock_name"] == "NIFTY-CE-24400-11Jul2024"].sort_values(by="datetime")

    portfolio = Portfolio()

    # Apply the function of trade logic to each row of the DataFrame
    data = [portfolio.trade(row.to_dict()) for _, row in df_TradeHistory.iterrows()]
    df_TradeHistory = pd.DataFrame(data)

    # Select columns that end with 'price' or 'amount'
    columns_to_round = [col for col in df_TradeHistory.columns if col.endswith("price") or col.endswith("amount")]
    # Round the values in the selected columns to two decimal places
    df_TradeHistory[columns_to_round] = df_TradeHistory[columns_to_round].round(2)
    df = df_TradeHistory.drop(columns=["exchange", "segment", "scrip_code", "stock_name", "amount", "price", "symbol"])

    return df


class Portfolio:
    def __init__(self):
        self.stocks = dict()

    def trade(self, record: dict = None):
        stock_name = str(record.get("stock_name"))

        if stock_name not in self.stocks:
            self.stocks[stock_name] = Stock(stock_name)

        print(
            str(record.get("datetime")),
            str(record.get("side")).upper().rjust(10, " "),
            str(record.get("quantity")).rjust(10, " "),
            str(record.get("price")).rjust(10, " "),
        )

        record.update(
            self.stocks[stock_name].trade(
                side=str(record.get("side")).upper(),
                price=float(record.get("price")),
                quantity=int(record.get("quantity")),
            )
        )
        return record


class Stock:
    def __init__(self, stock_name):
        self.stock_name = stock_name
        self.holding_price = 0
        self.holding_quantity = 0
        self.holding_amount = 0

    def trade(
        self,
        side: str,
        price: float,
        quantity: int,
    ):
        pnl_amount = 0
        if side == "BUY":
            buy_price = price
            buy_amount = quantity * price

            sell_price = 0
            sell_amount = quantity * sell_price

            self.holding_quantity += quantity
            self.holding_amount += buy_amount
        elif side == "SELL":
            sell_price = price
            sell_amount = quantity * price

            buy_price = self.holding_price
            buy_amount = quantity * buy_price

            self.holding_quantity -= quantity
            self.holding_amount -= buy_amount
            pnl_amount = sell_amount - buy_amount
        else:
            raise Exception(f"{side} was never excepected")
        
        
        
        
        if self.holding_quantity != 0:
            self.holding_price = self.holding_amount / self.holding_quantity
        else:
            self.holding_price = 0

        return {
            "quantity": quantity,
            "buy_price": buy_price,
            "buy_amount": buy_amount,
            "sell_price": sell_price,
            "sell_amount": sell_amount,
            "holding_price": self.holding_price,
            "holding_quantity": self.holding_quantity,
            "holding_amount": self.holding_amount,
            "pnl_amount": pnl_amount,
        }


main()

2024-07-09 12:44:04        BUY       25.0       89.0
2024-07-09 12:44:04        BUY      125.0       89.0
2024-07-09 13:17:26       SELL       50.0      117.1
2024-07-09 13:17:26       SELL      100.0      117.1
2024-07-11 11:05:32       SELL       25.0        4.3
2024-07-11 14:09:06        BUY       10.0        2.5
2024-07-11 14:09:06        BUY       15.0        2.5


,datetime,side,quantity,buy_price,buy_amount,sell_price,sell_amount,holding_price,holding_quantity,holding_amount,pnl_amount
0,2024-07-09 12:44:04,BUY,25,89.0,2225.0,0.0,0.0,89.00,25,2225.0,0.0
1,2024-07-09 12:44:04,BUY,125,89.0,11125.0,0.0,0.0,89.00,150,13350.0,0.0
2,2024-07-09 13:17:26,SELL,50,89.0,4450.0,117.1,5855.0,89.00,100,8900.0,1405.0
3,2024-07-09 13:17:26,SELL,100,89.0,8900.0,117.1,11710.0,0.00,0,0.0,2810.0
4,2024-07-11 11:05:32,SELL,25,0.0,0.0,4.3,107.5,-0.00,-25,0.0,107.5
5,2024-07-11 14:09:06,BUY,10,2.5,25.0,0.0,0.0,-1.67,-15,25.0,0.0
6,2024-07-11 14:09:06,BUY,15,2.5,37.5,0.0,0.0,0.00,0,62.5,0.0


In [17]:
df

,datetime,side,quantity,buy_price,buy_amount,sell_price,sell_amount,holding_price,holding_quantity,holding_amount,pnl_amount
0,2024-07-09 12:44:04,BUY,25,89.0,2225.0,0.0,0.0,89.0,25,2225.0,0
1,2024-07-09 12:44:04,BUY,125,89.0,11125.0,0.0,0.0,89.0,150,13350.0,0
2,2024-07-09 13:17:26,SELL,50,89.0,4450.0,117.1,5855.0,89.0,100,8900.0,1405
3,2024-07-09 13:17:26,SELL,100,89.0,8900.0,117.1,11710.0,0.0,0,0.0,2810
4,2024-07-11 11:05:32,SELL,25,0.0,0.0,4.3,107.5,4.3,-25,-107.5,0
5,2024-07-11 14:09:06,BUY,10,2.5,25.0,0.0,0.0,5.5,-15,-82.5,0
6,2024-07-11 14:09:06,BUY,15,2.5,37.5,0.0,0.0,0.0,0,0.0,-45
